In [3]:
!pip install numpy tensorflow scikit-learn

In [9]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.utils import resample

# Load datasets
train_data = np.load('/kaggle/input/train-test-data/train_data_partial.npy')
train_labels = np.load('/kaggle/input/train-test-data/train_labels_partial.npy')
test_data = np.load('/kaggle/input/train-test-data/test_data_partial.npy')
test_labels = np.load('/kaggle/input/train-test-data/test_labels_partial.npy')
member_data = np.load('/kaggle/input/train-test-data/test_member_data.npy')
non_member_data = np.load('/kaggle/input/train-test-data/test_Nonmember_data.npy')
evaluation_data = np.load('/kaggle/input/train-test-data/evaluation_data.npy')

# Normalize data and add the channel dimension if not already added (for grayscale images)
if len(train_data.shape) == 3:  # If data is (num_samples, height, width)
    train_data = train_data[..., np.newaxis]  # Shape: (num_samples, height, width, 1)
    test_data = test_data[..., np.newaxis]    # Shape: (num_samples, height, width, 1)

# Model Architecture
def create_model(input_shape):
    model = Sequential([
        tf.keras.Input(shape=input_shape),
        Conv2D(32, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    return model

# Compile Model
input_shape = train_data.shape[1:]  # Grayscale images (height, width, 1)
model = create_model(input_shape)
model.compile(optimizer=Adam(learning_rate=1e-3),
              loss=SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

# Train Model
model.fit(train_data, train_labels, epochs=10, validation_split=0.2, batch_size=64, verbose=1)

# Evaluate Model
test_accuracy = model.evaluate(test_data, test_labels, verbose=0)[1]
print(f"Clean test accuracy: {test_accuracy:.4f}")

# Backdoor Attack
print("Conducting Backdoor Attack...")
trigger = np.ones((3, 3, 1)) * 255.0 / 255.0  # Normalized trigger
test_data_with_trigger = test_data.copy()
test_data_with_trigger[:10, :3, :3, :] = trigger

# Evaluate Backdoor Effectiveness
backdoor_preds = model.predict(test_data_with_trigger[:10])
backdoor_effectiveness = np.mean(np.argmax(backdoor_preds, axis=1) == 0)  # Target class = 0
print(f"Backdoor effectiveness (target class 0): {backdoor_effectiveness:.4f}")

# Defense Against Backdoor
print("Defending Against Backdoor Attack...")
# Advanced methods like anomaly detection or adversarial pruning can be implemented here

# Poisoning Defense Using Bagging
print("Defending Against Poisoning Attack...")
aggregated_predictions = np.zeros((len(test_data), 10))

for i in range(10):  # Train 10 models with resampled subsets
    x_sample, y_sample = resample(train_data, train_labels, n_samples=len(train_data) // 2)
    poison_model = create_model(input_shape)
    poison_model.compile(optimizer=Adam(learning_rate=1e-3),
                         loss=SparseCategoricalCrossentropy(),
                         metrics=['accuracy'])
    poison_model.fit(x_sample, y_sample, epochs=5, verbose=0)
    predictions = poison_model.predict(test_data)
    aggregated_predictions += predictions

final_predictions = np.argmax(aggregated_predictions, axis=1)
bagging_accuracy = np.mean(final_predictions == test_labels)
print(f"Bagging defense accuracy: {bagging_accuracy:.4f}")

# Membership Inference Attack
print("Conducting Membership Inference Attack...")
shadow_model = create_model(input_shape)
shadow_model.compile(optimizer=Adam(learning_rate=1e-3),
                     loss=SparseCategoricalCrossentropy(),
                     metrics=['accuracy'])
shadow_model.fit(train_data, train_labels, epochs=5, verbose=1)

# Attack Dataset for Membership Inference
member_data = member_data[..., np.newaxis]  # Add channel dimension for input consistency
non_member_data = non_member_data[..., np.newaxis]
train_shadow_preds = shadow_model.predict(member_data)
test_shadow_preds = shadow_model.predict(non_member_data)
attack_x = np.concatenate([train_shadow_preds, test_shadow_preds], axis=0)
attack_y = np.concatenate([np.ones(len(member_data)), np.zeros(len(non_member_data))])

# Train Attack Model
attack_model = Sequential([
    Dense(64, activation='relu', input_shape=(10,)),
    Dense(1, activation='sigmoid')
])
attack_model.compile(optimizer=Adam(learning_rate=1e-3),
                     loss='binary_crossentropy',
                     metrics=['accuracy'])
attack_model.fit(attack_x, attack_y, epochs=5, batch_size=64, verbose=1)

# Evaluate Membership Inference Attack
attack_accuracy = attack_model.evaluate(attack_x, attack_y, verbose=0)[1]
print(f"Membership inference attack accuracy: {attack_accuracy:.4f}")


Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 10s 64ms/step - accuracy: 0.1850 - loss: 2.6880 - val_accuracy: 0.1225 - val_loss: 4.5348
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 7s 57ms/step - accuracy: 0.2802 - loss: 1.9815 - val_accuracy: 0.1425 - val_loss: 5.8650
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 7s 58ms/step - accuracy: 0.3364 - loss: 1.7878 - val_accuracy: 0.3065 - val_loss: 2.7746
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 7s 56ms/step - accuracy: 0.3860 - loss: 1.6697 - val_accuracy: 0.4190 - val_loss: 1.8313
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 11s 61ms/step - accuracy: 0.4086 - loss: 1.6065 - val_accuracy: 0.4650 - val_loss: 1.6012
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 7s 57ms/step - accuracy: 0.4393 - loss: 1.5330 - val_accuracy: 0.4670 - val_loss: 1.5825
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 7s 57ms/step - accuracy: 0.4682 - loss: 1.4505 - val_accuracy: 0.5010 - val_loss: 1.5000
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 7s 57ms/step - accuracy: 0.4948 - loss: 1.3831 - val_ac